# Log metrics with MLflow in PyTorch Lightning

description: log mlflow metrics in pytorch lightning with azureml as the backend tracking store

Lightning supports many popular [logging frameworks](https://pytorch-lightning.readthedocs.io/en/stable/loggers.html). [MLflow](https://mlflow.org/) is a popular open-source library for managing the lifecycle of your ML projects. Azure ML offers integration with MLflow, including for training. Specifically, Azure ML integrates as a backend tracking store for MLflow's [Tracking](https://mlflow.org/docs/latest/tracking.html#) component for logging metrics and managing runs. This tutorial will cover using the MLflow logger and leveraging the Azure ML MLflow integration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
# training script
source_dir = "src"
script_name = "train-with-mlflow-logging.py"

# environment file
environment_file = "environment.yml"

# azure ml settings
environment_name = "pt-lightning"
experiment_name = "pt-lightning-mlflow-tutorial"
compute_name = "gpu-K80-2"

## Create environment

Define a conda environment YAML file with your training script dependencies and create an Azure ML environment. This notebook will use the same environment definition that was used for part 1 of the tutorial. The dependencies include **mlflow** and **azureml-mlflow**, which are needed for logging with MLflow.

In [ ]:
from azureml.core import Environment

env = Environment.from_conda_specification(environment_name, environment_file)

# specify a GPU base image
env.docker.enabled = True
env.docker.base_image = (
    "mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04"
)

## Enable logging in training script

In *train_with_mlfow_logging.py*:

### 1. Create an MLFlowLogger
To configure the MLFlowLogger, you will need to provide the following:

* **Tracking URI**: Specify the tracking URI to point to your Azure ML Workspace in order to use Azure ML as the backend tracking store for MLflow. You can get the URI with mlflow.get_tracking_uri().
* **Experiment name**: Use the same name as the name of your Azure ML experiment.
* **Run ID**: You will need to link the MLFlowLogger's run ID to the ID of the Azure ML run.

To get the MLflow Run object of the training run, use the mlflow `mlflow.start_run()` method. Once you have the Run object, you can then access the experiment information.

```python
import mlflow
import os

with mlflow.start_run() as run:
    mlflow_uri = mlflow.get_tracking_uri()
    exp_id = run.info.experiment_id
    exp_name = mlflow.get_experiment(exp_id).name
    
    mlf_logger = MLFlowLogger(experiment_name=exp_name, tracking_uri=mlflow_uri)
    mlf_logger._run_id = run.info.run_id

    trainer = pl.Trainer.from_argparse_args(args, logger=mlf_logger)
```

Lightning will then take care of setting the tracking URI, creating the MLflow experiment, starting the MLflow run, and creating the underlying `MlflowClient` object.

    
### 2. Log metrics
You can then log metrics and other objects in your script. This tutorial's training script leverages Lightning's automatic log functionalities to log the loss metric by calling `self.log()` inside the `training_step()` method. Since logging too frequently can slow down training, the tutorial logs at the end of every epoch.

```python
self.log('loss', loss, on_epoch=True, on_step=False)
```

For more information on logging and the configurable options, see Lightning's [Logging](https://pytorch-lightning.readthedocs.io/en/stable/logging.html) documentation and the [MLFlowLogger](https://pytorch-lightning.readthedocs.io/en/stable/logging.html#mlflow) reference documentation.

### Configure and run training job

Create a ScriptRunConfig to specify the training script & arguments, environment, and cluster to run on.

In [ ]:
from azureml.core import ScriptRunConfig, Experiment

src = ScriptRunConfig(
    source_directory=source_dir,
    script=script_name,
    arguments=["--max_epochs", 25, "--gpus", 2, "--accelerator", "ddp"],
    compute_target=compute_name,
    environment=env,
)

run = Experiment(ws, experiment_name).submit(src)
run

If you navigate to the Azure ML studio UI, you can see the logged metrics visualized under the Experiment view and the "Metrics" tab of the individual Run view.

In [ ]:
run.wait_for_completion(show_output=True)